## Ingest races.csv file

Step 1 Read the csv file\
Step 2 Add the columns\
Step 3 Select the required columns\
Step 4 Rename the columns\
Step 5 Wrtie data to datalake as parquet

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

### Step 1 Read the csv file

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType


In [0]:
races_schema = StructType(fields=[StructField("raceId", IntegerType(), False),
                                  StructField("year", IntegerType(), True),
                                  StructField("round", IntegerType(), True),
                                  StructField("circuitId", IntegerType(), True),
                                  StructField("name", StringType(), True),
                                  StructField("date", DateType(), True),
                                  StructField("time", StringType(), True),
                                  StructField("url", StringType(), True)
])

In [0]:
races_df = spark.read\
    .option("header", True)\
        .schema(races_schema)\
            .csv(f"{raw_folder_path}/{v_file_date}/races.csv")

### **Step 2 Add the columns**

In [0]:
from pyspark.sql.functions import concat, lit, to_timestamp, col

In [0]:
races_with_timestamp_df = races_df\
    .withColumn("race_timestamp",\
         to_timestamp(concat(col('date'), lit(' '), col('time')), 'yyyy-MM-dd HH:mm:ss'))

In [0]:
races_with_ingestion_date_df = add_ingestion_date(races_with_timestamp_df)

### Step 3 Select the required columns

In [0]:
races_selected_df = races_with_ingestion_date_df.select(col("raceId"),col("year"),col("round"),col("circuitId"),col("name"),col("race_timestamp"),col("ingestion_date"))

### Step 4 Rename the columns

In [0]:
races_renamed_df = races_selected_df.withColumnRenamed("raceId", "race_id")\
    .withColumnRenamed("year", "race_year")\
        .withColumnRenamed("circuitId", "circuit_id")\
        .withColumn("data-source", lit(v_data_source))\
            .withColumn("file_date", lit(v_file_date))

In [0]:
display(races_renamed_df)

race_id,race_year,round,circuit_id,name,race_timestamp,ingestion_date,data-source,file_date
1,2009,1,1,Australian Grand Prix,2009-03-29T06:00:00Z,2025-11-19T20:09:37.178641Z,,2021-04-18
2,2009,2,2,Malaysian Grand Prix,2009-04-05T09:00:00Z,2025-11-19T20:09:37.178641Z,,2021-04-18
3,2009,3,17,Chinese Grand Prix,2009-04-19T07:00:00Z,2025-11-19T20:09:37.178641Z,,2021-04-18
4,2009,4,3,Bahrain Grand Prix,2009-04-26T12:00:00Z,2025-11-19T20:09:37.178641Z,,2021-04-18
5,2009,5,4,Spanish Grand Prix,2009-05-10T12:00:00Z,2025-11-19T20:09:37.178641Z,,2021-04-18
6,2009,6,6,Monaco Grand Prix,2009-05-24T12:00:00Z,2025-11-19T20:09:37.178641Z,,2021-04-18
7,2009,7,5,Turkish Grand Prix,2009-06-07T12:00:00Z,2025-11-19T20:09:37.178641Z,,2021-04-18
8,2009,8,9,British Grand Prix,2009-06-21T12:00:00Z,2025-11-19T20:09:37.178641Z,,2021-04-18
9,2009,9,20,German Grand Prix,2009-07-12T12:00:00Z,2025-11-19T20:09:37.178641Z,,2021-04-18
10,2009,10,11,Hungarian Grand Prix,2009-07-26T12:00:00Z,2025-11-19T20:09:37.178641Z,,2021-04-18


### Step 5 Wrtie data to datalake as parquet

In [0]:
#races_renamed_df.write.mode("overwrite").partitionBy('race_year').parquet(f"{processed_folder_path}/races")
races_renamed_df.write.mode("overwrite").format("delta").saveAsTable("f1_processed.races")

In [0]:
%sql
SELECT * FROM f1_processed.races;

race_id,race_year,round,circuit_id,name,race_timestamp,ingestion_date,data-source,file_date
1,2009,1,1,Australian Grand Prix,2009-03-29T06:00:00Z,2025-11-19T20:09:41.952806Z,,2021-04-18
2,2009,2,2,Malaysian Grand Prix,2009-04-05T09:00:00Z,2025-11-19T20:09:41.952806Z,,2021-04-18
3,2009,3,17,Chinese Grand Prix,2009-04-19T07:00:00Z,2025-11-19T20:09:41.952806Z,,2021-04-18
4,2009,4,3,Bahrain Grand Prix,2009-04-26T12:00:00Z,2025-11-19T20:09:41.952806Z,,2021-04-18
5,2009,5,4,Spanish Grand Prix,2009-05-10T12:00:00Z,2025-11-19T20:09:41.952806Z,,2021-04-18
6,2009,6,6,Monaco Grand Prix,2009-05-24T12:00:00Z,2025-11-19T20:09:41.952806Z,,2021-04-18
7,2009,7,5,Turkish Grand Prix,2009-06-07T12:00:00Z,2025-11-19T20:09:41.952806Z,,2021-04-18
8,2009,8,9,British Grand Prix,2009-06-21T12:00:00Z,2025-11-19T20:09:41.952806Z,,2021-04-18
9,2009,9,20,German Grand Prix,2009-07-12T12:00:00Z,2025-11-19T20:09:41.952806Z,,2021-04-18
10,2009,10,11,Hungarian Grand Prix,2009-07-26T12:00:00Z,2025-11-19T20:09:41.952806Z,,2021-04-18


In [0]:
dbutils.notebook.exit("Success")